In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/wine_data

import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from time import perf_counter
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score
import itertools
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/wine_data


##### DeepFM 패키지 사용

In [ ]:
pip install deepctr[gqu]

### 0. Data Load

In [ ]:
train_df = pd.read_json('train_all_meta_v2.json')
test_df = pd.read_json('test_all_meta_v2.json')

### 1. Data Preprocess & feature selection

In [ ]:
# # feature.v1 (epoch 10: 0.8476)
# X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','wine_id','userID','food','grapes','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','user_follower_count','user_following_count','user_rating_count','wine_id','userID','food','grapes','region_id']]
# y_test = test_df['like']

In [ ]:
# # feature.v2 (epoch 10: 0.8471)
# X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','wine_id','userID','food','grapes','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','winery_ratings_count','winery_ratings_average','wine_id','userID','food','grapes','region_id']]
# y_test = test_df['like']

In [ ]:
# # feature.v3 (epoch 10: 0.852)
# X_train = train_df.loc[:,['rating_count','rating_average','body','acidity','alcohol','wine_id','userID','food','grapes','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count','rating_average','body','acidity','alcohol', 'wine_id','userID','food','grapes','region_id']]
# y_test = test_df['like']

In [ ]:
# feature.v4 (epoch 10: 0.8539)
X_train = train_df.loc[:,['rating_count','rating_average','wine_id','userID','food','grapes','region_id']]
y_train = train_df['like']
X_test = test_df.loc[:,['rating_count','rating_average', 'wine_id','userID','food','grapes','region_id']]
y_test = test_df['like']

In [ ]:
# # feature.v5 (epoch 10: 0.8499)
# X_train = train_df.loc[:,['rating_average', 'wine_id','userID','food','grapes','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_average', 'wine_id','userID','food','grapes','region_id']]
# y_test = test_df['like']

In [ ]:
# # feature.v6 (epoch 10: 0.8515)
# X_train = train_df.loc[:,['rating_count', 'rating_average', 'wine_id','userID','food','grapes']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count', 'rating_average', 'wine_id','userID','food','grapes']]
# y_test = test_df['like']

In [ ]:
# # feature.v7 (epoch 10: 0.8529)
# X_train = train_df.loc[:,['rating_count','rating_average','wine_id','userID','grapes','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count','rating_average', 'wine_id','userID','grapes','region_id']]
# y_test = test_df['like']

In [ ]:
# # feature.v8 (epoch 10: 0.8509)
# X_train = train_df.loc[:,['rating_count','rating_average','wine_id','userID','food','region_id']]
# y_train = train_df['like']
# X_test = test_df.loc[:,['rating_count','rating_average', 'wine_id','userID','food','region_id']]
# y_test = test_df['like']

In [ ]:
# food 리스트 -> str
food_str = []
for i in range(len(X_train['food'])):
    if X_train['food'][i]:food_str.append(' '.join(X_train['food'][i]))
    else:food_str.append(' ')
X_train.food = pd.Series(food_str, name='food')

food_str = []
for i in range(len(X_test['food'])):
    if X_test['food'][i]:food_str.append(' '.join(X_test['food'][i]))
    else:food_str.append(' ')
X_test.food = pd.Series(food_str, name='food')

# grapes 리스트 -> str
grapes_str = []
for i in range(len(X_train['grapes'])):
    if X_train['grapes'][i]:grapes_str.append(' '.join(X_train['grapes'][i]))
    else:grapes_str.append(' ')
X_train.grapes = pd.Series(grapes_str, name='grapes')

grapes_str = []
for i in range(len(X_test['grapes'])):
    if X_test['grapes'][i]:grapes_str.append(' '.join(X_test['grapes'][i]))
    else:grapes_str.append(' ')
X_test.grapes = pd.Series(grapes_str, name='grapes')

# userID, region_id -> str
X_train.userID = X_train.userID.astype(str)
X_train.region_id = X_train.region_id.astype(str)
X_train.wine_id = X_train.wine_id.astype(str)

X_test.userID = X_test.userID.astype(str)
X_test.region_id = X_test.region_id.astype(str)
X_test.wine_id = X_test.wine_id.astype(str)

### 2. Data Setting

In [ ]:
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score

# feature.v1 .v4
sparse_features = ['userID', 'food', 'grapes', 'region_id', 'wine_id']

# feature.v6
# sparse_features = ['userID', 'food', 'grapes', 'wine_id']

# feature.v7
# sparse_features = ['userID', 'grapes', 'region_id', 'wine_id']

# feature.v8
# sparse_features = ['userID', 'food', 'region_id', 'wine_id']

# feature.v1
# dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol',
#        'winery_ratings_count', 'winery_ratings_average', 'user_follower_count',
#        'user_following_count', 'user_rating_count']

# feature.v2
# dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol',
#        'winery_ratings_count', 'winery_ratings_average']

# feature.v3
# dense_features = ['rating_count', 'rating_average', 'body', 'acidity', 'alcohol']

# feature.v4 .v6 .v7
dense_features = ['rating_count', 'rating_average']

# feature.v5
# dense_features = ['rating_average']


for feat in sparse_features:
        lbe = LabelEncoder()
        all = pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates() # train, test 전체를 묶어서 LabelEncoder fit
        lbe = lbe.fit(all)
        X_train[feat] = lbe.transform(X_train[feat])
        X_test[feat] = lbe.transform(X_test[feat])

mms = MinMaxScaler(feature_range=(0, 1))
X_train[dense_features] = mms.fit_transform(X_train[dense_features])
X_test[dense_features] = mms.transform(X_test[dense_features])

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=pd.concat([X_train[feat], X_test[feat]], axis=0).drop_duplicates().nunique(),embedding_dim=4)  # vocabulary_size를 train, test 전체로부터
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,) for feat in dense_features]  

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 결측치 mean으로 처리
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

train_model_input = {name:X_train[name] for name in feature_names}
test_model_input = {name:X_test[name] for name in feature_names}


### 3. Model

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

### 4. Train & Result

In [ ]:
history = model.fit(train_model_input, y_train.values,
                    batch_size=256, epochs=100, verbose=2, validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)
print("test AUC", round(roc_auc_score(y_test.values, pred_ans), 4))
print("\ntest Auccuracy", round(accuracy_score(y_test.values, pred_ans.round()), 4))

Epoch 1/100
2386/2386 - 28s - loss: 0.5689 - binary_crossentropy: 0.5671 - val_loss: 0.6557 - val_binary_crossentropy: 0.6527
Epoch 2/100
2386/2386 - 25s - loss: 0.5354 - binary_crossentropy: 0.5303 - val_loss: 0.6651 - val_binary_crossentropy: 0.6587
Epoch 3/100
2386/2386 - 24s - loss: 0.5173 - binary_crossentropy: 0.5090 - val_loss: 0.6715 - val_binary_crossentropy: 0.6622
Epoch 4/100
2386/2386 - 24s - loss: 0.5022 - binary_crossentropy: 0.4912 - val_loss: 0.7067 - val_binary_crossentropy: 0.6947
Epoch 5/100
2386/2386 - 24s - loss: 0.4897 - binary_crossentropy: 0.4761 - val_loss: 0.7329 - val_binary_crossentropy: 0.7182
Epoch 6/100
2386/2386 - 24s - loss: 0.4771 - binary_crossentropy: 0.4610 - val_loss: 0.7355 - val_binary_crossentropy: 0.7183
Epoch 7/100
2386/2386 - 24s - loss: 0.4636 - binary_crossentropy: 0.4453 - val_loss: 0.7284 - val_binary_crossentropy: 0.7091
Epoch 8/100
2386/2386 - 25s - loss: 0.4507 - binary_crossentropy: 0.4304 - val_loss: 0.7547 - val_binary_crossentropy:

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
userID (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
food (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
grapes (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
region_id (InputLayer)          [(None, 1)]          0                                            
______________________________________________________________________________________________

In [ ]:
from sklearn.metrics import classification_report

y_pred = np.around(pred_ans)
print(classification_report(y_test.values, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.78      0.79     88227
           1       0.81      0.81      0.81    100491

    accuracy                           0.80    188718
   macro avg       0.80      0.80      0.80    188718
weighted avg       0.80      0.80      0.80    188718

